<a href="https://colab.research.google.com/github/AndersonZangelmi/Exercicio_Python/blob/master/BD_API_GOV_AZ_OWNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
import pyspark
print(pyspark.__version__)

3.0.1


In [8]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: a656fa09df76dfd3de97a34a006c19f3' 'http://www.portaltransparencia.gov.br/api-de-dados/despesas/plano-orcamentario?ano=2019&pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5033    0  5033    0     0   8632      0 --:--:-- --:--:-- --:--:--  8632


In [9]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre as Viagens
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

In [10]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- codPOIdAcompanhamento: string (nullable = true)
 |-- codUnidadeOrcamentaria: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- codigoAcao: string (nullable = true)
 |-- codigoFuncao: string (nullable = true)
 |-- codigoPrograma: string (nullable = true)
 |-- codigoSubFuncao: string (nullable = true)
 |-- descPOIdAcompanhamento: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- id: long (nullable = true)
 |-- numAno: long (nullable = true)

+---------------------+----------------------+------+----------+------------+--------------+---------------+----------------------+--------------------+------+------+
|codPOIdAcompanhamento|codUnidadeOrcamentaria|codigo|codigoAcao|codigoFuncao|codigoPrograma|codigoSubFuncao|descPOIdAcompanhamento|           descricao|    id|numAno|
+---------------------+----------------------+------+----------+------------+--------------+---------------+----------------------+--------------------+------+------+
|     

In [11]:
dfPrepared = df.selectExpr("dataTransacao","estabelecimento.razaoSocialReceita","estabelecimento.nome","estabelecimento.municipio.uf.sigla","portador.nome as nome_do_portador","tipoCartao.descricao","valorTransacao")
dfPrepared.show()

AnalysisException: ignored

In [ ]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [ ]:
topTravels = spark.sql("select count(*),nome_do_portador as politico from dfPrepared group by 2 order by 1 desc")
topTravels.show()

+--------+--------------------+
|count(1)|            politico|
+--------+--------------------+
|       3|LYGIA MARIA MARIN...|
|       1|FLAVIO CARLOS DE ...|
|       1|    MARIA NECY ALVES|
|       1|NEWTON DE AZEVEDO...|
|       1|OSNY PEREIRA DA S...|
|       1|  RICARDO ANUNCIACAO|
|       1|JOAO REGINALDO DA...|
|       1|BRUNO MORAES DA S...|
|       1|  GIANO CUTRIM SILVA|
|       1|MATHEUS LINO FERR...|
|       1|MANOEL OLIMPIO NO...|
|       1|   ADILIO ALVES LEAL|
|       1|   JEAN ELAINE SILVA|
+--------+--------------------+

